In [1]:
import pandas as pd
from src.utils import load_config, save_df
from src.bronze_ingest import bronze_ingest
from src.silver_transform import silver_transform
from src.gold_enrich import gold_enrich

def main():
    # --- Load Config ---
    cfg = load_config("config/settings_test.yaml")  # or "config/settings_test.yaml"
    
    # --- Bronze Layer ---
    bronze_df = bronze_ingest(
        storis_path=cfg["data_paths"]["storis_csv"],
        afi_path=cfg["data_paths"]["afi_csv"]
    )
    print(f"✅ Bronze layer created with {len(bronze_df)} records")

    # --- Silver Layer ---
    silver_df = silver_transform(bronze_df)
    save_df(silver_df, cfg["data_paths"]["silver_output"])
    print("✅ Silver layer saved at:", cfg["data_paths"]["silver_output"])

    # --- Gold Layer ---
    gold_df = gold_enrich(
        df=silver_df,
        model=cfg["ollama"]["model"],
        system_prompt=cfg["ollama"]["system"],
        prompt_template=cfg["ollama"]["prompt"]
    )
    save_df(gold_df, cfg["data_paths"]["gold_output"])
    print("✅ Gold layer saved at:", cfg["data_paths"]["gold_output"])

if __name__ == "__main__":
    main()


c:\Users\abaker\OneDrive - Factory Direct of Edison\Desktop\textgen-pipeline\src\bronze_ingest.py:5: DtypeWarning: Columns (13,14,16,22,32,37,40,47,69,72,96,97,99,101,149,179,180,189,190,191,192,193,194,197,203,204,219,225,226,230,234,242,244) have mixed types. Specify dtype option on import or set low_memory=False.
  storis_data = pd.read_csv(storis_path)
c:\Users\abaker\OneDrive - Factory Direct of Edison\Desktop\textgen-pipeline\src\bronze_ingest.py:6: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  afi_data = pd.read_csv(afi_path)


✅ Bronze layer created with 10 records
✅ Silver layer saved at: data/silver/clean_products.csv


100%|██████████| 10/10 [31:47<00:00, 190.72s/it]

✅ Gold layer saved at: data/gold/final_products.csv
